In [7]:
import sys
sys.path.append("../")
import pandas as pd
import datetime as dt
from api.oanda_api import OandaApi
from dateutil import parser
import timeit
import plotly.graph_objects as go
import numpy as np
from technicals import trend_checker
from technicals import zone_detector
from technicals import pattern_detector
from scipy.signal import argrelextrema
pd.set_option('display.max_columns', None)

In [3]:
def apply_downtrend(df):
    df['ma_10'] = df.mid_c.rolling(window=10).mean()
    df['ma_50'] = df.mid_c.rolling(window=50).mean()
    df['ma_100'] = df.mid_c.rolling(window=100).mean()
    df['ma_150'] = df.mid_c.rolling(window=150).mean()
    df['ma_200'] = df.mid_c.rolling(window=200).mean()
    df['in_downtrend'] = df['ma_10'] < df['ma_150']
    df.dropna(inplace=True)

def detect_setup(
    df,
    strength_col='bullish_strength_score',
    strength_threshold=0.7,
    lookahead=25,
    proximity_pips=0.0030,
    rolling_window=40,
    breakout_threshold=20  # in pips
):
    df = df.copy().reset_index(drop=True)
    df['setup_stage'] = None

    # Step 1: Bottom detection
    df['is_bottom'] = (
        (df['mid_l'] == df['mid_l'].rolling(window=rolling_window).min()) &
        (df['in_downtrend'] == True)
    )
    df.loc[df['is_bottom'], 'setup_stage'] = 'bottom'

    # Step 2: Track active zone
    df['active_zone_low'] = None
    df['active_zone_high'] = None

    current_low = None
    current_high = None
    last_bottom_idx = None
    breakout_found = False
    reentry_found = False
    confirmation_found = False

    for i in range(len(df)):
        if df.at[i, 'is_bottom']:
            # New setup: reset everything
            current_low = df.at[i, 'mid_l']
            current_high = df.at[i, 'mid_h']
            last_bottom_idx = i
            breakout_found = False
            reentry_found = False
            confirmation_found = False

        df.at[i, 'active_zone_low'] = current_low
        df.at[i, 'active_zone_high'] = current_high

        # Skip if setup was invalidated
        if current_high is None:
            continue

        max_allowed_low = current_high + breakout_threshold / 10000.0

        # Invalidate if price goes too far from zone
        if breakout_found and df.at[i, 'mid_l'] > max_allowed_low:
            current_low = None
            current_high = None
            last_bottom_idx = None
            breakout_found = False
            reentry_found = False
            confirmation_found = False
            df.at[i, 'active_zone_low'] = None
            df.at[i, 'active_zone_high'] = None
            continue

        # Step 2: Breakout
        if not breakout_found and df.at[i, 'mid_l'] > current_high:
            df.at[i, 'setup_stage'] = 'breakout'
            breakout_found = True
            continue  # Reentry can't be on breakout candle

        # Step 3: Reentry
        if breakout_found and not reentry_found:
            if current_low <= df.at[i, 'mid_l'] <= current_high:
                df.at[i, 'setup_stage'] = 'reentry'
                reentry_found = True
                continue  # Confirmation comes after reentry

        # Step 4: Confirmation (with spacing from bottom)
        if breakout_found and reentry_found and not confirmation_found:
            if df.at[i, strength_col] > strength_threshold:
                if i - last_bottom_idx >= 6:  # ✅ Enforce minimum spacing from bottom
                    df.at[i, 'setup_stage'] = 'confirmation'
                    confirmation_found = True

    return df




def detect_bottom_reversal_setups(
    df,
    strength_col='bullish_strength_score',
    strength_threshold=0.7,
    lookahead=25,
    proximity_pips=0.0030,
    rolling_window=40,
    breakout_threshold=20  # in pips (e.g. 30 = 0.0030)
):
    """
    Detects bottom reversal setups using new lows that occur only during downtrends.

    A setup is:
    1. A bottom candle (lowest in a window, while in_downtrend is True)
    2. A breakout candle (low + high > zone high)
    3. A reentry candle (low reenters the zone but never breaks below zone low)
    4. A strong bullish candle (close near zone high and strength score high)

    The setup is invalidated if any candle after reentry has a low greater than
    breakout_threshold above the zone high.

    Returns:
        DataFrame with setup_stage column added.
    """
    df = df.copy().reset_index(drop=True)
    df['setup_stage'] = None

    # Step 1: Tag bottom candles
    df['is_bottom'] = (
        (df['mid_l'] == df['mid_l'].rolling(window=rolling_window).min()) &
        (df['in_downtrend'] == True)
    )

    df.loc[df['is_bottom'], 'setup_stage'] = 'bottom'
    bottom_indexes = df.index[df['is_bottom']].tolist()

    for i in bottom_indexes:
        # 🔥 Always reset the active zone on every new bottom
        active_zone_low = df.at[i, 'mid_l']
        active_zone_high = df.at[i, 'mid_h']
        active_bottom_idx = i

        # Step 2: Look for breakout
        breakout_idx = None
        for j in range(i + 1, len(df)):
            row_j = df.iloc[j]
            if row_j['mid_l'] < active_zone_low:
                break  # Setup invalidated by lower low
            if row_j['mid_l'] > active_zone_high and row_j['mid_h'] > active_zone_high:
                breakout_idx = j
                break

        if breakout_idx is None or breakout_idx - i > lookahead:
            continue

        # Step 3: Reentry
        reentry_idx = None
        for k in range(breakout_idx + 1, breakout_idx + lookahead):
            if k >= len(df):
                break
            row_k = df.iloc[k]
            if active_zone_low <= row_k['mid_l'] <= active_zone_high:
                reentry_idx = k
                break
            if row_k['mid_l'] < active_zone_low:
                break  # Invalidated by new low

        if reentry_idx is None:
            continue

        # Step 4: Confirmation + breakout distance check
        confirmation_found = False
        breakout_distance_limit = active_zone_high + breakout_threshold / 10000.0
        df['breakout_distance_limit'] = breakout_distance_limit
        df['active_zone_high'] = active_zone_high
        df['breakout_threshold'] = breakout_threshold / 10000.0
        for m in range(reentry_idx + 1, reentry_idx + lookahead):
            if m >= len(df):
                break
            row_m = df.iloc[m]

            if row_m['mid_l'] < active_zone_low:
                break  # Invalidated by new low
            if row_m['mid_l'] > breakout_distance_limit:
                break  # Price went too far — invalidated

            if row_m[strength_col] >= strength_threshold:
                if abs(row_m['mid_l'] - active_zone_high) <= proximity_pips:
                    df.at[breakout_idx, 'setup_stage'] = 'breakout'
                    df.at[reentry_idx, 'setup_stage'] = 'reentry'
                    df.at[m, 'setup_stage'] = 'confirmation'
                    confirmation_found = True
                    break

        # No need to manually reset active_* vars — loop resets on next bottom

    return df


def find_support_resistance(df, price_col='mid_c', high_col='mid_h', low_col='mid_l', window=3, clustering_threshold=0.0050):
    """
    Identifies support and resistance levels in candlestick data.
    
    Args:
        df (pd.DataFrame): Your OHLC dataframe.
        price_col (str): Column name for close/mid price.
        high_col (str): Column name for highs.
        low_col (str): Column name for lows.
        window (int): Lookback window to detect local highs/lows.
        clustering_threshold (float): Maximum distance between levels to consider them the same zone.

    Returns:
        Tuple[List[float], List[float]]: (support_levels, resistance_levels)
    """
    
    local_min_idx = argrelextrema(df[low_col].values, np.less_equal, order=window)[0]
    local_max_idx = argrelextrema(df[high_col].values, np.greater_equal, order=window)[0]

    raw_supports = df.iloc[local_min_idx][low_col].values
    raw_resistances = df.iloc[local_max_idx][high_col].values

    def cluster_levels(levels):
        clustered = []
        levels = sorted(levels)
        for level in levels:
            if not clustered:
                clustered.append([level])
            elif abs(level - np.mean(clustered[-1])) <= clustering_threshold:
                clustered[-1].append(level)
            else:
                clustered.append([level])
        return [round(np.mean(group), 5) for group in clustered if len(group) >= 2]  # Only return stronger levels

    support_levels = cluster_levels(raw_supports)
    resistance_levels = cluster_levels(raw_resistances)

    return support_levels, resistance_levels

def get_zones_for_price(price, support_levels, resistance_levels, num_of_zones=3, min_gap=0.0, min_width=0.0015):
    """
    Returns non-overlapping (support, resistance) zones where the support is above the given price,
    and there's at least `min_gap` space and `min_width` size.

    Args:
        price (float): Current price.
        support_levels (list of float): Detected support levels.
        resistance_levels (list of float): Detected resistance levels.
        num_of_zones (int): Number of zones to return.
        min_gap (float): Minimum gap between zones.
        min_width (float): Minimum acceptable width of a zone.

    Returns:
        List of tuples: [(support1, resistance1), (support2, resistance2), ...]
    """

    support_levels = sorted(support_levels)
    resistance_levels = sorted(resistance_levels)

    zones = []
    last_resistance = price

    sup_above = [s for s in support_levels if s > price]

    for support in sup_above:
        if support <= last_resistance + min_gap:
            continue

        possible_resistances = [r for r in resistance_levels if r > support]
        for resistance in possible_resistances:
            width = resistance - support
            if width >= min_width:
                zones.append((support, resistance))
                last_resistance = resistance
                break  # Move on to the next zone

        if len(zones) == num_of_zones:
            break

    return zones


def attach_zones_to_confirmations(
    df,
    window=3,
    clustering_threshold=0.0050,
    num_of_zones=3
):
    """
    For each confirmation candle:
        - Attach relevant support/resistance zones based on past data only
        - Compute zone-to-stop-loss ratio using second zone
        - Add 'confirmation_zones', 'zone_sl_ratio', and 'meets_ratio' columns
    """
    from copy import deepcopy
    df['confirmation_zones'] = None
    df['zone_sl_ratio'] = None
    df['meets_ratio'] = False

    for i in range(len(df)):
        if df.at[i, 'setup_stage'] == 'confirmation':
            past_df = df.iloc[:i]
            if len(past_df) < window * 2:
                continue

            support_levels, resistance_levels = find_support_resistance(
                past_df,
                price_col='mid_c',
                high_col='mid_h',
                low_col='mid_l',
                window=window,
                clustering_threshold=clustering_threshold
            )

            current_price = df.at[i, 'mid_c']
            current_low = df.at[i, 'mid_l']

            zones = get_zones_for_price(
                price=current_price,
                support_levels=support_levels,
                resistance_levels=resistance_levels,
                num_of_zones=num_of_zones
            )

            df.at[i, 'confirmation_zones'] = deepcopy(zones)

            if len(zones) >= 2:
                zone_top = zones[1][1]  # Top of second zone (resistance)
                reward = zone_top - current_price
                risk = current_price - current_low

                if risk > 0:
                    ratio = reward / risk
                    df.at[i, 'zone_sl_ratio'] = round(ratio, 3)
                    df.at[i, 'meets_ratio'] = ratio >= 1.0

def apply_technicals(df):
    df['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df.time]
    apply_downtrend(df)
    df['bullish_strength_score'] = [bullish_strength_with_context(df, i) for i in range(len(df))]
    #need to make this not return a df but add in place
    df = detect_setup(df)
    attach_zones_to_confirmations(df)
    return df

def simulate_trades(df):
    df['trade'] = None
    df['pips'] = None

    pip_size = 0.0001
    active_trade = None

    for i in range(len(df)):
        row = df.iloc[i]

        # Manage active trade
        if active_trade:
            high = row['mid_h']
            low = row['mid_l']
            price = row['mid_c']

            # SL hit
            if low <= active_trade['stop_loss']:
                df.at[i, 'trade'] = 'closed - sl hit'
                pnl = (active_trade['stop_loss'] - active_trade['entry_price']) / pip_size
                df.at[i, 'pips'] = round(pnl, 1)
                active_trade = None
                continue

            # TP hit
            if high >= active_trade['take_profit']:
                df.at[i, 'trade'] = 'closed - tp hit'
                pnl = (active_trade['take_profit'] - active_trade['entry_price']) / pip_size
                df.at[i, 'pips'] = round(pnl, 1)
                active_trade = None
                continue

        # Entry condition
        if not active_trade and row['setup_stage'] == 'confirmation' and row['meets_ratio']:
            if isinstance(row['confirmation_zones'], list) and len(row['confirmation_zones']) >= 2:
                # Find the linked bottom candle
                bottom_idx = df.index[:i][df['setup_stage'][:i] == 'bottom'].max()
                bottom_candle = df.loc[bottom_idx]
                entry_price = row['ask_c']
                stop_loss = bottom_candle['mid_l'] - 0.0005
                take_profit = row['confirmation_zones'][1][1]  # top of zone 2

                active_trade = {
                    'entry_idx': i,
                    'entry_price': entry_price,
                    'stop_loss': stop_loss,
                    'take_profit': take_profit,
                    'zones': row['confirmation_zones'],
                }

                df.at[i, 'trade'] = 'opened'

    return df

def plot_trade_zones(df, title="Trade Zones & Entries"):
    fig = go.Figure()

    y_min = df['mid_l'].min()
    y_max = df['mid_h'].max()
    padding = (y_max - y_min) * 0.1

    # Candlesticks
    fig.add_trace(go.Candlestick(
        x=df['sTime'],
        open=df['mid_o'],
        high=df['mid_h'],
        low=df['mid_l'],
        close=df['mid_c'],
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B', 
        decreasing_fillcolor='#CC2E3C',
        increasing_line_color='#24A06B',
        decreasing_line_color='#FF3A4C',
        name="Candles"
    ))

    # Add MAs
    # fig.add_trace(go.Scatter(
    #     x=df['sTime'],
    #     y=df['ma_10'],
    #     line=dict(width=2),
    #     line_shape='spline',
    #     name='MA_10'
    # ))
    # fig.add_trace(go.Scatter(
    #     x=df['sTime'],
    #     y=df['ma_150'],
    #     line=dict(width=2),
    #     line_shape='spline',
    #     name='MA_150'
    # ))

    # Mark confirmation (entry) candles
    entry_df = df[df['setup_stage'] == 'confirmation']
    fig.add_trace(go.Candlestick(
        x=entry_df['sTime'],
        open=entry_df['mid_o'],
        high=entry_df['mid_h'],
        low=entry_df['mid_l'],
        close=entry_df['mid_c'],
        line=dict(width=2), opacity=1,
        increasing_fillcolor='yellow',
        increasing_line_color='yellow'
        
    ))

    # Mark bottom candles
    bottom_df = df[df['setup_stage'] == 'bottom']
    fig.add_trace(go.Scatter(
        x=bottom_df['sTime'],
        y=bottom_df['mid_l'],
        mode='markers',
        marker=dict(color='green', size=15, symbol='triangle-up'),
        name='Bottm',
        showlegend=True
    ))

    # Mark reentry candles
    reentry_df = df[df['setup_stage'] == 'reentry']
    fig.add_trace(go.Scatter(
        x=reentry_df['sTime'],
        y=reentry_df['mid_h'],
        mode='markers',
        marker=dict(color='blue', size=15, symbol='triangle-down'),
        name='Reentry',
        showlegend=True
    ))

    for i, row in entry_df.iterrows():
        zones = row['confirmation_zones']  # This should be a list of tuples
        time = row['sTime']

        for zone in zones:
            low, high = zone
            fig.add_trace(go.Scatter(
                x=[time, time],
                y=[low, high],
                mode='lines',
                line=dict(color='yellow', width=2, dash='dot'),
                name='Zone'  # Show legend only once
            ))

    # Mark downtrend candles
    # downtrend_df = df[df['in_downtrend'] == True]
    # fig.add_trace(go.Candlestick(
    #     x=downtrend_df['sTime'],
    #     open=downtrend_df['mid_o'],
    #     high=downtrend_df['mid_h'],
    #     low=downtrend_df['mid_l'],
    #     close=downtrend_df['mid_c'],
    #     line=dict(width=1), opacity=1,
    #     increasing_fillcolor='yellow',
    #     increasing_line_color='yellow',
    #     decreasing_line_color='yellow',
    #     name="Candles"
    # ))

    # Layout
    fig.update_layout(
        title=title,
        xaxis_title='Time',
        yaxis_title='Price',
        width=2000,
        height=1000,
        hovermode='x unified',
        margin=dict(l=10, r=10, b=10, t=30),
        paper_bgcolor="#2c303c",
        plot_bgcolor="#2c303c",
        font=dict(size=10, color="#e1e1e1"),
        yaxis=dict(
            range=[y_min - padding, y_max + padding],
            fixedrange=False  # Allow zooming
        )
    )

    fig.update_xaxes(
        gridcolor="#1f292f",
        rangeslider=dict(visible=True),
        nticks=5
    )

    fig.update_yaxes(
        gridcolor="#1f292f"
    )

    fig.show()

def filter_df_by_date(df, start, end):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    return df[(df['time'] >= start) & (df['time'] <= end)]
    
def bullish_strength_with_context(df, index, lookback=200):
    row = df.iloc[index]
    
    open_price = row['mid_o']
    close_price = row['mid_c']
    high = row['mid_h']
    low = row['mid_l']

    # 1. Must be bullish
    if close_price <= open_price:
        return 0.0

    body = close_price - open_price
    total_range = high - low

    #2. Body must make up most of candle
    if body < (total_range * 0.7):
        return 0.0

    # 3. Relative size vs past lookback of candles
    start = max(index - lookback, 0)
    candle_sizes = []
    for i in range(start, index):
        row_i = df.iloc[i]
        body_i = row_i['mid_h'] - row_i['mid_l']
        candle_sizes.append(body_i)

    if not candle_sizes:
        return 0.0

    # Percentile rank of this candle's body size
    candle_avg = np.mean(candle_sizes)
    if body >= candle_avg:
        return 1.0
    else:
        return round((body / candle_avg) * 0.9, 3)

def summarize_trades(filepath):

    df = pd.read_pickle(filepath)

    # Filter only closed trades
    closed_trades = df[df['trade'].isin(['closed - sl hit', 'closed - tp hit'])].copy()

    if closed_trades.empty:
        print("No closed trades found.")
        return

    # Calculate metrics
    total_trades = len(closed_trades)
    total_pips = closed_trades['pips'].sum()
    win_rate = (closed_trades['trade'] == 'closed - tp hit').mean() * 100

    # Cumulative profit over time
    closed_trades['cumulative_pips'] = closed_trades['pips'].cumsum()
    closed_trades['time'] = pd.to_datetime(closed_trades['time'])

    # Print summary
    print(f"📈 Total Trades: {total_trades}")
    print(f"💰 Total Pips: {round(total_pips, 1)}")
    print(f"✅ Win Rate: {round(win_rate, 2)}%")

    # Plot using Plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=closed_trades['time'],
        y=closed_trades['cumulative_pips'],
        mode='lines+markers',
        name='Cumulative Pips',
        line=dict(width=2),
        marker=dict(size=4)
    ))

    fig.update_layout(
        title="📈 Strategy Profit Over Time",
        xaxis_title="Time",
        yaxis_title="Cumulative Pips",
        template="plotly_white",
        hovermode="x unified"
    )

    fig.show()

In [4]:
df = pd.read_pickle('../data/EUR_USD_H4.pkl')
# df = filter_df_by_date(df, "2022-01-01T00:00:00Z", "2023-01-01T00:00:00Z")
df = apply_technicals(df)
df = simulate_trades(df)
# df_between.head(45)

In [5]:
# USED TO VIEW EACH TRADE ONE AT A TIME LIKE THIS plot_trade_zone(df_trades[0])
df_trades = []
pre_context = 40

# Get indexes of trade entries
entry_indexes = df.index[df['trade'] == 'opened'].tolist()

for entry_idx in entry_indexes:
    # Find the most recent 'bottom' before this trade
    subset = df.loc[:entry_idx]
    bottom_idx = subset[subset['setup_stage'] == 'bottom'].last_valid_index()
    if bottom_idx is None:
        continue

    # Look ahead to find where this trade closes
    close_idx = None
    for j in range(entry_idx + 1, len(df)):
        if df.at[j, 'trade'] in ('closed - sl hit', 'closed - tp hit'):
            close_idx = j
            break

    if close_idx is None:
        continue  # trade never closed in the available data

    start_idx = max(bottom_idx - pre_context, 0)
    trade_df = df.iloc[start_idx:close_idx + 1].copy()
    df_trades.append(trade_df)

print(f'total trades: { len(df_trades) }')


total trades: 15


In [ ]:
for i, trade_df in enumerate(df_trades):
    print(f"Plotting trade #{i + 1}")
    plot_trade_zones(trade_df)

In [ ]:
df_between = df[(df['sTime'] >= "s22-04-11 00:00") & (df['sTime'] <= "s22-04-13 00:00")]
df_between.head(60)

In [57]:
plot_trade_zones(df)

In [17]:
pairs = ["USD_JPY"]
for pair in pairs:
    print(pair)
    summarize_trades(f"../backtesting/results/{pair}_H1_analyzed.pkl")

USD_JPY
📈 Total Trades: 5
💰 Total Pips: -22.0
✅ Win Rate: 60.0%


In [14]:
pairs = ["AUD_USD", "EUR_USD", "GBP_USD", "USD_CHF", "USD_JPY", "NZD_USD", "USD_CAD"]
for pair in pairs:
    print(pair)
    summarize_trades(f"../backtesting/results/{pair}_H1_analyzed.pkl")

AUD_USD


FileNotFoundError: [Errno 2] No such file or directory: '../backtesting/results/AUD_USD_H1_analyzed.pkl'

In [ ]:
df_an = pd.read_pickle('../data/GBP_USD_H4.pkl').copy()
def apply_technicals(df):
    df['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df.time]
    trend_checker.apply_downtrend(df)
    df['bullish_strength_score'] = df.apply(pattern_detector.bullish_strength, axis=1)
    #need to make this not return a df but add in place
    df = pattern_detector.detect_bottom_reversal_setups(df)
    zone_detector.attach_zones_to_confirmations(df)
    return df

df_an = apply_technicals(df_an)
df_an.head(40)

In [ ]:
df_an['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df_an.time]
trend_checker.apply_downtrend(df_an)
df_an.tail(20)
support_levels, resistance_levels = zone_detector.find_support_resistance(df_an)
zones = zone_detector.get_zones_for_price(1.01934, support_levels, resistance_levels)
print(zones)

In [ ]:
df_bull = df_an.copy()
df_bull['bullish_strength_score'] = df_bull.apply(pattern_detector.bullish_strength, axis=1)
strength_threshold = 0.7
df_bull['strong_bullish'] = df_bull['bullish_strength_score'] >= strength_threshold
df_bull.head()


In [ ]:
df_signals = pattern_detector.detect_bottom_reversal_setups(df_bull)
zone_detector.attach_zones_to_confirmations(df_signals)
df_signals = df_signals[df_signals['setup_stage'] == 'confirmation']
df_signals.shape

In [ ]:
fig = trend_checker.highlight_downtrend_candles(df_an)
x_start = df_an['sTime'].iloc[0]
x_end = df_an['sTime'].iloc[-1]
y_min = df_an['mid_l'].min()
y_max = df_an['mid_h'].max()
padding = (y_max - y_min) * 0.1  # 10% vertical padding

stage_colors = {
    'bottom': 'blue',
    'breakout': 'orange',
    'reentry': 'purple',
    'confirmation': 'green'
}

# Add a trace for each setup stage
for _, row in df_signals.iterrows():
    stage = row['setup_stage']
    fig.add_trace(go.Scatter(
        x=[row['sTime']],
        y=[row['mid_c']],  # or mid_l if you want the low of the candle
        mode='markers+text',
        marker=dict(color=stage_colors.get(stage, 'black'), size=10, symbol='x'),
        name=stage,
        text=[stage],
        textposition='top center',
        showlegend=False  # Set to True if you want to display multiple legends
    ))

# # Plot support levels
# for level in support_levels:
#     fig.add_trace(go.Scatter(
#         x=[x_start, x_end],
#         y=[level, level],
#         mode='lines',
#         name=f'Support {level}',
#         line=dict(color='green', width=1)
#     ))

#     # Plot resistance levels
# for level in resistance_levels:
#     fig.add_trace(go.Scatter(
#         x=[x_start, x_end],
#         y=[level, level],
#         mode='lines',
#         name=f'Resistance {level}',
#         line=dict(color='red', width=1)
#     ))

fig.update_layout(
    yaxis=dict(
        range=[y_min - padding, y_max + padding],
        fixedrange=False  # Allow zooming
    )
)

fig.show()